In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
df_abt_reg = pd.read_csv('/content/drive/MyDrive/datasets/preco_casas_final.csv', sep=';')
df_abt_reg.head()

,zoneamento,tam_terreno,forma_terreno,qualidade_geral,condicao,ano_construcao,qualidade_aquecedor,ar_condicionado,tam_primeiro_andar,tam_segundo_andar,...,qtde_banheiros,qtde_comodos,qtde_lareiras,qtde_carros_garagem,tam_garagem,tam_piscina,qualidade_piscina,mes_venda,ano_venda,preco
0,RL,785.03035,Reg,7,5,2003,Ex,Y,79.524968,79.339162,...,30,8,0,2,50.910844,0.0,NaN,2,2008,846510.0
1,RL,891.86880,Reg,6,8,1976,Ex,Y,117.243586,0.000000,...,2,6,1,2,42.735380,0.0,NaN,5,2007,NaN
2,RL,1045.15875,IR1,7,5,2001,Ex,Y,85.470760,80.453998,...,2,6,1,2,56.485024,0.0,NaN,9,2008,907410.0
3,RL,887.22365,IR1,7,5,1915,Gd,Y,89.279783,70.234668,...,1,7,1,3,59.643726,0.0,NaN,2,2006,568400.0
4,RL,1324.79678,IR1,8,5,2000,Ex,Y,106.373935,97.826859,...,2,9,1,3,77.666908,0.0,NaN,12,2008,1015000.0


In [4]:
!pip install feature-engine -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 3.4 MB/s eta 0:00:00


In [5]:
cat_vars = []
num_vars = ['tam_terreno', 'qualidade_geral', 'ano_construcao', 'tam_primeiro_andar', 'tam_segundo_andar', 'tam_sala_estar',
            'qtde_banheiros',	'qtde_comodos',	'qtde_lareiras',	'qtde_carros_garagem',	'tam_garagem', 'ano_venda']
modeling_vars = cat_vars + num_vars

df_abt_reg = df_abt_reg[modeling_vars + ['preco']].dropna()


X = df_abt_reg[modeling_vars].copy()
y = df_abt_reg['preco'].copy()

In [6]:
df_abt_reg.isnull().sum()

tam_terreno            0
qualidade_geral        0
ano_construcao         0
tam_primeiro_andar     0
tam_segundo_andar      0
tam_sala_estar         0
qtde_banheiros         0
qtde_comodos           0
qtde_lareiras          0
qtde_carros_garagem    0
tam_garagem            0
ano_venda              0
preco                  0
dtype: int64


## LinearRegression

Os principais hiperparâmetros que podemos utilizar no GridSearch:

* `fit_intercept`: Se deve ou não computar o intercepto. Se definido como `False`, não será computado o intercepto.

In [7]:
import time
t1 = time.time()

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[('linear_regression', LinearRegression())])

parametros = {
  'linear_regression__fit_intercept': [True, False],
}

grid_search = GridSearchCV(pipeline, parametros, scoring='neg_median_absolute_error', cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

print()
print('='*100)
print(grid_search.best_params_)
print('\nDemorou {} segundos para GridSearch para LinearRegression'.format(time.time() - t1))
print('='*100)
print()

Fitting 5 folds for each of 2 candidates, totalling 10 fits

{'linear_regression__fit_intercept': False}

Demorou 3.6128597259521484 segundos para GridSearch para LinearRegression




## SGDRegressor

Os principais hiperparâmetros que podemos utilizar no GridSearch:

* `fit_intercept`: Se deve ou não computar o intercepto. Se definido como `False`, não será computado o intercepto.

* `learning_rate`: Taxa de aprendizado, valores para testar: `['constant', 'optimal', 'adaptive']`.

* `penalty`: Regularização a ser utilizada: `['l2', 'l1', None]`

* `early_stopping`: Se usar a parada antecipada para encerrar o treinamento quando a pontuação de validação não estiver melhorando: `[True, False]`

* `n_iter_no_change`: Número de iterações sem melhorias antes de encerrar o treinamento: `[5, 30, 50]`



In [8]:
import time
t1 = time.time()

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[('sdg_reg', SGDRegressor())])

parametros = {
  'sdg_reg__fit_intercept': [True, False],
  'sdg_reg__learning_rate': ['constant', 'optimal', 'adaptive'],
  'sdg_reg__penalty': ['l2', 'l1', None],
  'sdg_reg__early_stopping': [True, False],
  'sdg_reg__n_iter_no_change': [5, 30, 50]
}

grid_search = GridSearchCV(pipeline, parametros, scoring='neg_median_absolute_error', cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X, y)

print()
print('='*100)
print(grid_search.best_params_)
print('\nDemorou {} segundos para GridSearch para SGDRegressor'.format(time.time() - t1))
print('='*100)
print()

Fitting 5 folds for each of 108 candidates, totalling 540 fits

{'sdg_reg__early_stopping': True, 'sdg_reg__fit_intercept': False, 'sdg_reg__learning_rate': 'adaptive', 'sdg_reg__n_iter_no_change': 30, 'sdg_reg__penalty': 'l1'}

Demorou 50.88233709335327 segundos para GridSearch para SGDRegressor

